In [1]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import os

Using TensorFlow backend.


In [2]:
def load_data() -> Tuple [np.ndarray, np.ndarray, 
                          np.ndarray, np.ndarray]:
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train / 255.
    x_test = x_test / 255.
    y_train = to_categorical(y_train, num_classes=10)
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = load_data()

In [3]:
print('x_train shape: {} | y_train shape: {}\nx_test shape : {} | y_test shape : {}'.format(x_train.shape, y_train.shape, 
                                                                                            x_test.shape, y_test.shape))

x_train shape: (50000, 32, 32, 3) | y_train shape: (50000, 10)
x_test shape : (10000, 32, 32, 3) | y_test shape : (10000, 1)


In [4]:
input_shape = x_train[0,:,:,:].shape
model_input = Input(shape=input_shape)

In [5]:
def conv_pool_cnn(model_input: Tensor) -> training.Model:
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='conv_pool_cnn')
    
    return model

In [6]:
conv_pool_cnn_model = conv_pool_cnn(model_input)

In [7]:
NUM_EPOCHS = 20
def compile_and_train(model: training.Model, num_epochs: int) -> Tuple [History, str]: 
    
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True,
                                                 save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train, y=y_train, batch_size=32, 
                     epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    weight_files = glob.glob(os.path.join(os.getcwd(), 'weights/*'))
    weight_file = max(weight_files, key=os.path.getctime) # most recent file
    return history, weight_file

In [8]:
_, conv_pool_cnn_weight_file = compile_and_train(conv_pool_cnn_model, NUM_EPOCHS)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 25s 620us/step - loss: 2.0245 - acc: 0.2202 - val_loss: 1.7044 - val_acc: 0.3624
Epoch 2/20
40000/40000 [==============================] - 24s 594us/step - loss: 1.5494 - acc: 0.4228 - val_loss: 1.3485 - val_acc: 0.5012
Epoch 3/20
40000/40000 [==============================] - 24s 592us/step - loss: 1.2703 - acc: 0.5398 - val_loss: 1.2550 - val_acc: 0.5450
Epoch 4/20
40000/40000 [==============================] - 24s 588us/step - loss: 1.0590 - acc: 0.6224 - val_loss: 0.9637 - val_acc: 0.6593
Epoch 5/20
40000/40000 [==============================] - 23s 583us/step - loss: 0.9154 - acc: 0.6742 - val_loss: 0.9065 - val_acc: 0.6838
Epoch 6/20
40000/40000 [==============================] - 23s 584us/step - loss: 0.8032 - acc: 0.7168 - val_loss: 0.8454 - val_acc: 0.7083
Epoch 7/20
40000/40000 [==============================] - 23s 583us/step - loss: 0.7133 - acc: 0.7471 - val_loss: 0.

In [9]:
def evaluate_error(model: training.Model) -> np.float64:
    pred = model.predict(x_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]   
 
    return error
(1 - evaluate_error(conv_pool_cnn_model)) * 100

73.47

In [10]:
def all_cnn(model_input: Tensor) -> training.Model:
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
        
    model = Model(model_input, x, name='all_cnn')
    
    return model
all_cnn_model = all_cnn(model_input)
_, all_cnn_weight_file = compile_and_train(all_cnn_model, NUM_EPOCHS)


print('Error for model 2:', 1 - evaluate_error(all_cnn_model))

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 24s 589us/step - loss: 1.7788 - acc: 0.3217 - val_loss: 1.5477 - val_acc: 0.4187
Epoch 2/20
40000/40000 [==============================] - 22s 559us/step - loss: 1.3593 - acc: 0.4954 - val_loss: 1.1841 - val_acc: 0.5686
Epoch 3/20
40000/40000 [==============================] - 22s 561us/step - loss: 1.0798 - acc: 0.6113 - val_loss: 0.9682 - val_acc: 0.6497
Epoch 4/20
40000/40000 [==============================] - 22s 559us/step - loss: 0.8921 - acc: 0.6825 - val_loss: 0.8621 - val_acc: 0.7003
Epoch 5/20
40000/40000 [==============================] - 22s 561us/step - loss: 0.7627 - acc: 0.7315 - val_loss: 0.8102 - val_acc: 0.7144
Epoch 6/20
40000/40000 [==============================] - 22s 559us/step - loss: 0.6574 - acc: 0.7691 - val_loss: 0.7312 - val_acc: 0.7453
Epoch 7/20
40000/40000 [==============================] - 22s 559us/step - loss: 0.5697 - acc: 0.7989 - val_loss: 0.

In [11]:
def nin_cnn(model_input: Tensor) -> training.Model:
    
    #mlpconv block 1
    x = Conv2D(32, (5, 5), activation='relu',padding='valid')(model_input)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block2
    x = Conv2D(64, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block3
    x = Conv2D(128, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='nin_cnn')
    
    return model

nin_cnn_model = nin_cnn(model_input)


_, nin_cnn_weight_file = compile_and_train(nin_cnn_model, NUM_EPOCHS)

print('Error for model 3:', 1 - evaluate_error(all_cnn_model))

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 7s 176us/step - loss: 1.8819 - acc: 0.2921 - val_loss: 1.6315 - val_acc: 0.3950
Epoch 2/20
40000/40000 [==============================] - 7s 163us/step - loss: 1.5991 - acc: 0.4081 - val_loss: 1.4645 - val_acc: 0.4568
Epoch 3/20
40000/40000 [==============================] - 6s 159us/step - loss: 1.4690 - acc: 0.4650 - val_loss: 1.3669 - val_acc: 0.5036
Epoch 4/20
40000/40000 [==============================] - 7s 163us/step - loss: 1.3949 - acc: 0.4920 - val_loss: 1.3263 - val_acc: 0.5159
Epoch 5/20
40000/40000 [==============================] - 7s 163us/step - loss: 1.3434 - acc: 0.5125 - val_loss: 1.2787 - val_acc: 0.5361
Epoch 6/20
40000/40000 [==============================] - 6s 162us/step - loss: 1.2970 - acc: 0.5293 - val_loss: 1.2211 - val_acc: 0.5532
Epoch 7/20
40000/40000 [==============================] - 6s 161us/step - loss: 1.2523 - acc: 0.5499 - val_loss: 1.2226 - 

In [12]:
conv_pool_cnn_model = conv_pool_cnn(model_input)
all_cnn_model = all_cnn(model_input)
nin_cnn_model = nin_cnn(model_input)

conv_pool_cnn_model.load_weights(conv_pool_cnn_weight_file)
all_cnn_model.load_weights(all_cnn_weight_file)
nin_cnn_model.load_weights(nin_cnn_weight_file)

models = [conv_pool_cnn_model, all_cnn_model, nin_cnn_model]


# The ensemble classifier
def ensemble(models: List [training.Model], model_input: Tensor) -> training.Model:
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

ensemble_model = ensemble(models, model_input)


print('Error for ensemble model:', 1 - evaluate_error(ensemble_model))

Error for ensemble model: 0.7874
